## Настройки Colab

In [1]:
# Настройка пользователя (сделать один раз)
!git config --global user.email "nabludatellip@gmail.com"
!git config --global user.name "ProninPV"

In [2]:
!git clone https://github.com/ProninPV/ml-regression_concrete-strength.git
%cd ml-regression_concrete-strength

Cloning into 'ml-regression_concrete-strength'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (432/432), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 432 (delta 228), reused 403 (delta 200), pack-reused 0 (from 0)
Receiving objects: 100% (432/432), 3.03 MiB | 8.29 MiB/s, done.
Resolving deltas: 100% (228/228), done.
/content/ml-regression_concrete-strength


In [3]:
%cd /content/ml-regression_concrete-strength
!git config pull.rebase false
!git pull origin tuning

/content/ml-regression_concrete-strength
From https://github.com/ProninPV/ml-regression_concrete-strength
 * branch            tuning     -> FETCH_HEAD
Updating a511430..77b46de
Fast-forward
 config/config.yaml                                 |   51 +
 ...0251115_121552.csv => modeling_experiments.csv} |    0
 .../pipelines/best_pipeline_LinearRegression.pkl   |  Bin 1249 -> 0 bytes
 .../pipeline_metadata_LinearRegression.pkl         |  Bin 1717 -> 0 bytes
 notebooks/05_Modeling_colab.ipynb                  | 1581 ++++++++++++++++----
 notebooks/06_Tuning.ipynb                          | 1260 ++++++++++++++++
 notebooks/EDA_step.txt                             |   73 +
 .../data/processed/eda_data_20251001_1007.csv      |  782 ----------
 .../data/processed/eda_data_20251001_1007.parquet  |  Bin 29101 -> 0 bytes
 .../data/processed/eda_data_20251001_1007.pkl      |  Bin 76022 -> 0 bytes
 .../data/processed/eda_data_20251001_1010.csv      |  782 ----------
 .../data/processed/eda_data_2

In [4]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.6 MB/s eta 0:00:00


In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.2 MB/s eta 0:00:00


## 6.0 Импорты библиотек

In [6]:
import os
import yaml
import logging
import pickle
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from scipy.stats import ttest_ind
from typing import List, Any, Optional, Tuple, Dict, Union
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import psutil
from tqdm import tqdm
import gc
import optuna
import joblib

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [10]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [11]:
# В Colab проект клонируется в /content/
# Устанавливаем правильную рабочую директорию
project_root = Path('/content/ml-regression_concrete-strength')

# Определяем корень проекта
# cwd = Path().resolve()
# project_root = cwd.parent

# Добавляем корень проекта в sys.path (этого достаточно)
sys.path.append(str(project_root))

# Проверяем наличие конфиг файла
config_path = project_root / "config" / "config.yaml"
print(f"Looking for config at: {config_path}")

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing
from src.modeling import modeling

# Передаем путь явно
config = loader.load_config(config_path)
print("✅ Config loaded successfully!")

Looking for config at: /content/ml-regression_concrete-strength/config/config.yaml
✅ Config loaded successfully!


In [12]:
!ls

catboost_info  docs	  README.md				 src
config	       models	  requirements.txt			 tests
data	       notebooks  skillbox_regression_project_notes.txt


## 6.1. Загрузка данных

In [13]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train',
                                         config=config)

[⧗] Загружаю данные из: /content/ml-regression_concrete-strength/data/processed/eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [14]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [15]:
# Загрузка отчета по экспериментам
modeling_result = modeling.load_sorted_modeling_report(config)
modeling_result

,outlier_strategy,model_name,model_type,dataset_size,mean_rmse,std_rmse,training_time_sec,memory_used_mb
0,['abnormal'],CatBoostRegressor,trees_models,770,4.2595,0.3446,13.49,0.44
1,['abnormal'],CatBoostRegressor,linear_models,770,4.2608,0.3868,11.55,12.38
2,"['gost_binar', 'combine']",CatBoostRegressor,linear_models,770,4.3196,0.4173,13.77,0.21
3,"['gost_binar', 'combine']",CatBoostRegressor,trees_models,770,4.3260,0.3906,14.72,0.03
4,['combine'],CatBoostRegressor,linear_models,770,4.3304,0.4210,11.68,0.15
...,...,...,...,...,...,...,...,...
105,['abnormal'],ElasticNet,trees_models,770,8.2662,0.4285,1.57,0.00
106,"['gost_binar', 'combine']",ElasticNet,linear_models,770,8.3014,0.4700,1.47,0.00
107,['gost_binar'],ElasticNet,linear_models,781,8.3183,0.5227,1.38,0.00
108,['abnormal'],ElasticNet,linear_models,770,8.3673,0.4635,1.29,0.00


In [16]:
# Определяем лучшую стратию
best_strategy = modeling.get_best_model_strategy(modeling_result)

best_model = best_strategy['model']
best_model_type = best_strategy['model_type']
best_rmse = best_strategy['rmse']
best_outlier_strategy = eval(best_strategy['outlier_strategy'])

In [17]:
best_strategy

{'model': 'CatBoostRegressor',
 'model_type': 'trees_models',
 'outlier_strategy': "['abnormal']",
 'rmse': np.float64(4.2595)}

## 6.2. Предобработка данных

In [18]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

In [19]:
# Конфигурация для преобразования признаков
feature_config = {
    'trend_settings': {
        'names': ['Linear', 'Log', 'Sqrt']  # Только безопасные
    }
}

In [20]:
# Обрабатываем выбросы для лучшей стратегии
outlier_handler = feat_preprocessing.OutlierHandler(
    strategies=best_outlier_strategy,
    config=config,
    target_col=y_name
)

X_processed, y_processed = outlier_handler.fit_transform(X, y)

In [21]:
X_processed

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,0.768743,0.032923
...,...,...,...,...,...,...,...,...,...,...
793,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,3,0.619355,0.000000
794,213.5,0.0,174.2,159.2,11.7,1043.6,771.9,100,0.745667,0.054801
795,304.8,0.0,99.6,196.0,9.8,959.4,705.2,28,0.643045,0.032152
797,288.0,192.0,0.0,192.0,0.0,932.0,717.8,28,0.666667,0.000000


## 6.2. Подбор гиперпараметров для CatboostRegressor

#### Определение фнукций

In [23]:
def create_cv_strategy(config: Dict[str, Any]):
    """
    Создает стратегию кросс-валидации из конфига.
    """
    cv_config = config['cross_validation']
    if cv_config['strategy'] == 'RepeatedKFold':
        return RepeatedKFold(
            n_splits=cv_config['n_splits'],
            n_repeats=cv_config['n_repeats'],
            random_state=cv_config['random_state']
        )
    else:
        return KFold(n_splits=cv_config['n_splits'], shuffle=True, random_state=cv_config['random_state'])

In [24]:
class ProgressBarCallback:
    def __init__(self, n_trials: int):
        self.n_trials = n_trials
        self.pbar = tqdm(total=n_trials, desc="Optuna Trials", unit="trial")
        self.completed_trials = 0

    def __call__(self, study: optuna.Study, trial: optuna.Trial):
        # Обновляем только при завершении trial
        if trial.state.is_finished():
            self.completed_trials += 1
            self.pbar.update(1)
            self.pbar.set_postfix({
                'best_rmse': f"{study.best_value:.4f}" if study.best_value else "N/A",
                'completed': self.completed_trials
            })

        if self.completed_trials >= self.n_trials:ф
            self.pbar.close()

#### Catboost

In [22]:
def create_catboost_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для CatBoost на основе конфига Optuna.
    """
    params = {}
    param_space = config['catboost_params']['search_space']

    # Динамически создаем параметры из search_space
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])
        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

    # Добавляем фиксированные параметры
    params.update(config['catboost_params']['fixed_params'])

    return params

In [53]:
# Проверяем, что лучшая модель - CatBoost
if best_strategy['model'] != 'CatBoostRegressor':
    print(f"⚠️  Лучшая модель не CatBoost, а {best_strategy['model']}")

print("🎯 Начинаем тюнинг лучшей CatBoost модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_catboost_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    params['allow_writing_files'] = False

    # Создаем модель CatBoost с отключенным выводом
    params['verbose'] = False
    model = CatBoostRegressor(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"catboost_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ
best_params['verbose'] = False

best_model = CatBoostRegressor(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

🎯 Начинаем тюнинг лучшей CatBoost модели:
   Модель: CatBoostRegressor
   Стратегия выбросов: ['abnormal']
   Базовый RMSE: 4.2595
⏳ Запускаем Optuna...
RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)


  0%|          | 0/100 [00:00<?, ?it/s]


✅ Тюнинг завершен!
📊 Результаты:
   До тюнинга: 4.2595
   После тюнинга: 3.9809
   Улучшение: 0.2786 (6.5%)


In [54]:
import joblib

# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/catboostregression_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_catboostregression_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

['models/tuning_models/catboostregression_3_9809.pkl']

💾 Модель сохранена: models/tuning_models/catboostregression_3_9809.pkl


['models/pipelines/pipeline_catboostregression_3_9809.pkl']

💾 Пайплайн сохранен: models/pipelines/pipeline_catboostregression_3_9809.pkl


## Отправка на Githubm

In [58]:
from getpass import getpass

# 1. Безопасный ввод токена
GITHUB_TOKEN = getpass('Введите ваш GitHub Personal Access Token: ')

Введите ваш GitHub Personal Access Token: ··········


In [55]:
!git status

On branch modeling
Your branch is ahead of 'origin/modeling' by 38 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   catboost_info/catboost_training.json
	modified:   catboost_info/learn/events.out.tfevents
	modified:   catboost_info/learn_error.tsv
	modified:   catboost_info/time_left.tsv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	models/pipelines/pipeline_catboostregression_3_9809.pkl
	models/tuning_models/

no changes added to commit (use "git add" and/or "git commit -a")


In [56]:
# 1. Добавляем файлы
!git add .

# 2. Коммитим
!git commit -m "fit: add tuning catboost regression"

[modeling 137dffd] fit: add tuning catboost regression
 6 files changed, 7680 insertions(+), 306 deletions(-)
 rewrite catboost_info/catboost_training.json (95%)
 rewrite catboost_info/learn/events.out.tfevents (94%)
 rewrite catboost_info/learn_error.tsv (93%)
 rewrite catboost_info/time_left.tsv (96%)
 create mode 100644 models/pipelines/pipeline_catboostregression_3_9809.pkl
 create mode 100644 models/tuning_models/catboostregression_3_9809.pkl


In [67]:
!git push https://{GITHUB_TOKEN}@github.com/ProninPV/ml-regression_concrete-strength.git tuning

Everything up-to-date


In [66]:
!git branch -a

  modeling
* tuning
  remotes/origin/HEAD -> origin/modeling
  remotes/origin/main
  remotes/origin/modeling
  remotes/origin/tuning


In [68]:
!git status

On branch tuning
Your branch is up to date with 'origin/tuning'.

nothing to commit, working tree clean
